In [9]:
# Exercise 1 #

import socket
import datetime

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('localhost', 65432))

message = input("Enter message: ")
client_socket.sendall(message.encode())
senTime= datetime.datetime.now()

response = client_socket.recv(1024)
recTime= datetime.datetime.now()
print(f"Server response: {response.decode()}")
print(recTime-senTime)

client_socket.close()

Server response: ACK: hello
0:00:00


In [10]:
# Exercise 2 #

import socket
import datetime

client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = ('localhost', 65432)

message = input("Enter message: ")

senTime = datetime.datetime.now()
client_socket.sendto(message.encode(), server_address)

response, server_address = client_socket.recvfrom(1024)
recTime = datetime.datetime.now()

print(f"Server response: {response.decode()}")
print(recTime-senTime)

client_socket.close()


Server response: ACK: hello
0:00:00.001024


In [46]:
# Exercise 3 #

import socket
import datetime

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('localhost', 65432))

with open('file_to_send.txt', 'rb') as f:
    file_data = f.read()
    senTime = datetime.datetime.now()

# Send the file data to the server
client_socket.sendall(file_data)

response = client_socket.recv(1024)
recTime= datetime.datetime.now()
print(f"Server response: {response.decode()}")
print(recTime-senTime)

client_socket.close()

Server response: ACK: file received
0:00:00.003929


When the file_to_send contains 'abrarur' it takes 2000 microseconds on average to receive response.

When the file_to_send contains two paragraphs of lorem ipsum it takes 2500 microseconds on average to recive response.

The averages were measures by sending the file 3 times for each contents.

In [11]:
# Exercise 4 #

import socket
import datetime

client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = ('localhost', 65432)

with open('file_to_send.txt', 'rb') as f:
    file_data = f.read()
    senTime = datetime.datetime.now()

client_socket.sendto(file_data, server_address)

response, server_address = client_socket.recvfrom(1024)
recTime = datetime.datetime.now()

print(f"Server response: {response.decode()}")
print(recTime-senTime)

client_socket.close()

Server response: ACK: file received
0:00:00.004098


The time needed to receive a response is 4200 microseconds on average, which is surprisingly slower than the time needed for the TCP client.

In [ ]:
# Exercise 6 and 7 #

def encrypt(msg, key):
    result= ''

    for char in msg:
        if (char.isupper()):
            result += chr((ord(char) + key-65) % 26 + 65)
        elif char.islower():
            result += chr((ord(char) + key - 97) % 26 + 97)
        else:
            result += char

    return result

import socket

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('localhost', 65432))

while True:
    message = input("Enter message: ")

    if message.lower() == 'exit':
        print("Exiting client.")
        break
    
    message = encrypt(message, 5)
    client_socket.sendall(message.encode())

    response = client_socket.recv(1024)
    print(f"Server response: {response.decode()}")

client_socket.close()

Server response: ACK: hello
Server response: ACK: hi
Server response: ACK: wazzup
Exiting client.


In [7]:
# Exercise 8 #

import socket
import requests

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect(('localhost', 65432))

api_url_uni = "https://api.open-meteo.com/v1/forecast?latitude=51.47&longitude=-0.0363&current_weather=true"
response_uni = requests.get(api_url_uni)

if response_uni.status_code == 200:
    weather_data_uni = response_uni.json()
    temperature_uni = weather_data_uni["current_weather"]["temperature"]

    message = f"Current temperature at uni: {temperature_uni}°C."
    message = encrypt(message, 5)
else:
    message = "Failed to fetch weather data"


client_socket.sendall(message.encode())

response = client_socket.recv(1024)
print(f"Server response: {response.decode()}")

client_socket.close()

Server response: ACK: Current temperature at uni: 9.2°C.
